In [1]:
import pandas as pd

In [2]:
def getUnitPreferences():
    unitTemp = ''
    while ((unitTemp != 'F') and (unitTemp != 'C')):
        print("-----------------------------------------")
        print("Would you like to enter your temperature in\n"
            "Fahrenheit (F) or Celcius (C). Enter F or C:", end=' ')
        unitTemp = input()
        if ((unitTemp != 'F') and (unitTemp != 'C')):
            print('\nInvalid Input')
        elif (unitTemp == 'F'):
            unitTempFull = 'Fahrenheit'     
        else:
            unitTempFull = 'Celcius'

    unitSnow = ''
    while ((unitSnow != 'cm') and (unitSnow != 'in')):            
        print("\n-----------------------------------------")
        print("Would you like to enter your snowfall in\n"
            "Centimeters (cm) or Inches (in). Enter cm or in:", end=' ')
        unitSnow = input()
        if ((unitSnow != 'cm') and (unitSnow != 'in')):
            print('\nInvalid Input')
        elif (unitSnow == 'cm'):
            unitSnowFull = 'Centimeters'     
        else:
            unitSnowFull = 'Inches'

    unitRain = ''
    while ((unitRain != 'cm') and (unitRain != 'in')):            
        print("\n-----------------------------------------")
        print("Would you like to enter your rainfall in\n"
            "Centimeters (cm) or Inches (in). Enter cm or in:", end=' ')
        unitRain = input()
        if ((unitRain != 'cm') and (unitRain != 'in')):
            print('\nInvalid Input')
        elif (unitRain == 'cm'):
            unitRainFull = 'Centimeters'     
        else:
            unitRainFull = 'Inches'

    return [unitTempFull, unitSnowFull, unitRainFull]

In [3]:
def getWeatherPreferences():
      print("-----------------------------------------")
      print("Welcome to the European Livability Index!\n"
            "The goal of this program is to take in your\n"
            "living preferences and provide you with a\n"
            "list of places you might enjoy!\n\n"
            "Currently the program creates your livability\n"
            "matching score based on desired temperature,\n"
            "rainfall, and snowfall for every season")

      unitChoices = getUnitPreferences()  

      print("\n-----------------------------------------")
      print("Enter your desired temperature in " + unitChoices[0] + " for every\n"
            "season in this format Spring Summer Fall Winter")
      print("-----------------------------------------")
      print("Enter choices here:")
      temperature = input().split(" ")
      print("-----------------------------------------")
      print("Enter your desired snowfall in " + unitChoices[1] + " for every\n"
            "season in this format Spring Summer Fall Winter")
      print("-----------------------------------------")
      print("Enter choices here:")
      snow = input().split(" ")
      print("-----------------------------------------")
      print("Enter your desired rainfall in " + unitChoices[2] + " for every\n"
            "season in this format Spring Summer Fall Winter")
      print("-----------------------------------------")
      print("Enter choices here:")
      rain = input().split(" ")
      print("-----------------------------------------")

      return [temperature, snow, rain], unitChoices


In [28]:
def normalizeDifference(actual, preference, sIndex):
    difference = abs(actual - preference[sIndex])
    min = difference.min()
    max = difference.max()
    differenceNormalized = (difference - min) / (max - min)
    return differenceNormalized

In [3]:
def computerWeatherScore(choices, unitChoices):

    weatherData = pd.read_csv("../../Data/Weather/weatherData.csv")
    springWeather = weatherData[weatherData['SEASON'] == 'Spring']
    summerWeather = weatherData[weatherData['SEASON'] == 'Summer']
    fallWeather = weatherData[weatherData['SEASON'] == 'Fall']
    winterWeather = weatherData[weatherData['SEASON'] == 'Winter']
    weatherDataSeasons = [springWeather, summerWeather, fallWeather, winterWeather]

    if (unitChoices[0] == 'Fahrenheit'):
        for index in range(len(choices[0])):
            choices[0][index] = (float(choices[0][index]) - 32.0) * (5/9)
    if (unitChoices[1] == 'Inches'):
        for index in range(len(choices[1])):
            choices[1][index] = (float(choices[1][index]) * 2.54)
        choices[1] = choices[1]
    if (unitChoices[2] == 'Inches'):
        for index in range(len(choices[2])):
            choices[2][index] = (float(choices[2][index]) * 2.54)

    scoreData = pd.DataFrame()
    sIndex = 0

    for season in weatherDataSeasons:
        season['SCORE'] = (3* normalizeDifference(season['Average Temp (C)'], choices[0], sIndex)) + normalizeDifference(season['Average Snow Depth (cm)'], choices[1], sIndex) + normalizeDifference(season['Average Rainfall (cm)'], choices[2], sIndex)  
        scoreData = pd.concat([scoreData, season])
        sIndex+=1
   
    return scoreData

    

In [ ]:
choices, unitChoices = getWeatherPreferences()

In [23]:
choices = [[70, 80, 65, 50],
           [2, 0, 9, 10],
           [36, 4, 9, 30]]
unitChoices = ['Fahrenheit', 'Inches', 'Inches']


In [ ]:
scores = computerWeatherScore(choices, unitChoices)
scoresDropped = scores.drop(columns=['SEASON', 'Average Temp (C)', 'Average Snow Depth (cm)', 'Average Rainfall (cm)'])
scoresRegion = scoresDropped.groupby(['MUNICIPALITY', 'COUNTRY']).agg({'SCORE': 'mean'})
scoresRegion = scoresRegion.sort_values(by=['SCORE'])

In [27]:
def computeTestResults(choices, unitChoices):   
    weatherData = pd.read_csv("../../Data/Weather/weatherData.csv")
    weatherData = weatherData[weatherData['MUNICIPALITY'] == 'Podgorica Municipality']
    springWeather = weatherData[weatherData['SEASON'] == 'Spring']
    summerWeather = weatherData[weatherData['SEASON'] == 'Summer']
    fallWeather = weatherData[weatherData['SEASON'] == 'Fall']
    winterWeather = weatherData[weatherData['SEASON'] == 'Winter']
    weatherDataSeasons = [springWeather, summerWeather, fallWeather, winterWeather]

    if (unitChoices[0] == 'Fahrenheit'):
        for index in range(len(choices[0])):
            choices[0][index] = (float(choices[0][index]) - 32.0) * (5/9)
    if (unitChoices[1] == 'Inches'):
        for index in range(len(choices[1])):
            choices[1][index] = (float(choices[1][index]) * 2.54)
        choices[1] = choices[1]
    if (unitChoices[2] == 'Inches'):
        for index in range(len(choices[2])):
            choices[2][index] = (float(choices[2][index]) * 2.54)

    scoreData = pd.DataFrame()
    sIndex = 0

    for season in weatherDataSeasons:
        season['SCORE'] = (3* normalizeDifference(season['Average Temp (C)'], choices[0], sIndex)) + normalizeDifference(season['Average Snow Depth (cm)'], choices[1], sIndex) + normalizeDifference(season['Average Rainfall (cm)'], choices[2], sIndex)  
        scoreData = pd.concat([scoreData, season])
        sIndex+=1

    return scoreData

In [13]:
scoresRegion.iloc[:10]

,,SCORE
MUNICIPALITY,COUNTRY,
Podgorica Municipality,Montenegro,0.722851
Primorje-Gorski Kotar County,Croatia,0.941491
Dubrovnik-Neretva County,Croatia,1.004961
Split-Dalmatia County,Croatia,1.008205
Zadar County,Croatia,1.093084
Nikšić Municipality,Montenegro,1.098033
Ticino,Switzerland,1.143834
Lazio,Italy,1.145637
Surxondaryo Region,Uzbekistan,1.160571


In [14]:
weatherData = pd.read_csv("../../Data/Weather/weatherData.csv")
weatherData[weatherData['MUNICIPALITY'] == 'Podgorica Municipality']

,SEASON,MUNICIPALITY,COUNTRY,Average Temp (C),Average Snow Depth (cm),Average Rainfall (cm)
140,Fall,Podgorica Municipality,Montenegro,16.956024,0.000000,6.459521
386,Spring,Podgorica Municipality,Montenegro,15.923962,0.000000,3.942702
631,Summer,Podgorica Municipality,Montenegro,27.594737,0.245390,1.662447
876,Winter,Podgorica Municipality,Montenegro,7.841942,0.487805,6.091870


In [ ]:
test = computeTestResults(choices, unitChoices)
test

In [38]:
choices = [[70, 80, 65, 50],
           [2, 0, 9, 10],
           [36, 4, 9, 30]]
unitChoices = ['Fahrenheit', 'Inches', 'Inches']

In [39]:
choices

[[70, 80, 65, 50], [2, 0, 9, 10], [36, 4, 9, 30]]

In [41]:
weatherData = pd.read_csv("../../Data/Weather/weatherData.csv")
weatherData = weatherData[weatherData['MUNICIPALITY'] == 'Podgorica Municipality']
springWeather = weatherData[weatherData['SEASON'] == 'Spring']
summerWeather = weatherData[weatherData['SEASON'] == 'Summer']
fallWeather = weatherData[weatherData['SEASON'] == 'Fall']
winterWeather = weatherData[weatherData['SEASON'] == 'Winter']
weatherDataSeasons = [springWeather, summerWeather, fallWeather, winterWeather]

if (unitChoices[0] == 'Fahrenheit'):
    for index in range(len(choices[0])):
        choices[0][index] = (float(choices[0][index]) - 32.0) * (5/9)
if (unitChoices[1] == 'Inches'):
    for index in range(len(choices[1])):
        choices[1][index] = (float(choices[1][index]) * 2.54)
    choices[1] = choices[1]
if (unitChoices[2] == 'Inches'):
    for index in range(len(choices[2])):
        choices[2][index] = (float(choices[2][index]) * 2.54)

print(choices)

scoreData = pd.DataFrame()
sIndex = 0

for season in weatherDataSeasons:
    season['SCORE'] = (3* normalizeDifference(season['Average Temp (C)'], choices[0], sIndex)) + normalizeDifference(season['Average Snow Depth (cm)'], choices[1], sIndex) + normalizeDifference(season['Average Rainfall (cm)'], choices[2], sIndex)  
    print(season)
    scoreData = pd.concat([scoreData, season])
    sIndex+=1

[[-6.049382716049383, -2.9629629629629624, -7.592592592592592, -12.222222222222223], [12.9032, 0.0, 58.0644, 64.51599999999999], [232.2576, 25.8064, 58.0644, 193.548]]
     SEASON            MUNICIPALITY     COUNTRY  Average Temp (C)  \
386  Spring  Podgorica Municipality  Montenegro         15.923962   

     Average Snow Depth (cm)  Average Rainfall (cm)  SCORE  
386                      0.0               3.942702    NaN  
     SEASON            MUNICIPALITY     COUNTRY  Average Temp (C)  \
631  Summer  Podgorica Municipality  Montenegro         27.594737   

     Average Snow Depth (cm)  Average Rainfall (cm)  SCORE  
631                  0.24539               1.662447    NaN  
    SEASON            MUNICIPALITY     COUNTRY  Average Temp (C)  \
140   Fall  Podgorica Municipality  Montenegro         16.956024   

     Average Snow Depth (cm)  Average Rainfall (cm)  SCORE  
140                      0.0               6.459521    NaN  
     SEASON            MUNICIPALITY     COUNTRY  Av

C:\Users\austi\AppData\Local\Temp\ipykernel_9616\2452899595.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season['SCORE'] = (3* normalizeDifference(season['Average Temp (C)'], choices[0], sIndex)) + normalizeDifference(season['Average Snow Depth (cm)'], choices[1], sIndex) + normalizeDifference(season['Average Rainfall (cm)'], choices[2], sIndex)
C:\Users\austi\AppData\Local\Temp\ipykernel_9616\2452899595.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season['SCORE'] = (3* normalizeDifference(s